In [1]:
# with open('dataset/raw/polyA_cs.fasta') as f:
#     lines = f.readlines()
#     for line in lines[:2]:
#         print(line)
#     print(len(lines))

In [2]:
!pip install --upgrade scikit-learn

Requirement already up-to-date: scikit-learn in /opt/conda/lib/python3.6/site-packages (0.22.1)


In [3]:
import sys
import numpy as np
import random
import argparse
from textwrap import dedent
from time import strftime
from itertools import product
import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Embedding, Bidirectional, Flatten, Dropout, GRU, Input, Conv1D, AveragePooling1D
from keras.models import Model
from keras import optimizers
from sklearn.model_selection import KFold
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix
import sklearn

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
# One hot encoding for bases
classes = {'A': 0,  # 0001
         'T': 1,  # 0010
         'C': 2,  # 0100
         'G': 3,  # 1000 
        }

def fasta_to_vectors(in_fasta):
    with open(in_fasta) as f:
        header_seq = f.readlines()
    
    seq = [header_seq[i * 2 + 1].strip().upper() for i in range(int(len(header_seq)/2))] # extract seq data only
    print(np.array(seq).shape)
    for s in seq:
        assert len(s) == 200
        continue
    
    seqs = np.zeros((len(seq), 200, 1))
    for i, s in enumerate(seq):
        for j, char in enumerate(s):
            seqs[i, j] = classes[char]
    output = to_categorical(seqs, num_classes=len(classes))
    return output


In [5]:
lab1_fasta = '../dataset/raw/polyA_cs.fasta' # args.polya
lab2_fasta = '../dataset/raw/non-polyA_cs.fasta' # args.cs
lab3_fasta = '../dataset/raw/non-cs.fasta'
l = 200

k = np.array([4, 6, 8, 10])

seq_vector_lab1 = np.array(fasta_to_vectors(lab1_fasta))
seq_vector_lab2 = np.array(fasta_to_vectors(lab2_fasta))
seq_vector_lab3 = np.array(fasta_to_vectors(lab3_fasta))
lab_vector_lab1 = np.tile([1, 0, 0], (len(seq_vector_lab1), 1))
lab_vector_lab2 = np.tile([0, 1, 0], (len(seq_vector_lab2), 1))
lab_vector_lab3 = np.tile([0, 0, 1], (len(seq_vector_lab3), 1))

(52457,)
(27595,)
(52460,)


In [6]:
# Build the whole model
random.seed(123)

size1 = len(seq_vector_lab1)
size2 = len(seq_vector_lab2)
size3 = len(seq_vector_lab3)

if size1 != size2 or size1 != size3:
    train_i1 = random.sample(range(size1), int(0.7 * size1))
    train_i2 = random.sample(range(size2), int(0.7 * size2))
    train_i3 = random.sample(range(size3), int(0.7 * size3))

else:
    train_i1 = random.sample(range(size1), int(0.7 * size1))
    train_i2 = train_i1
    train_i3 = train_i1

test_val1 = [i for i in range(size1) if i not in train_i1]
val_i1 = random.sample(test_val1, int(0.2 * size1))

test_val2 = [i for i in range(size2) if i not in train_i2]
val_i2 = random.sample(test_val2, int(0.2 * size2))

test_val3 = [i for i in range(size3) if i not in train_i3]
val_i3 = random.sample(test_val3, int(0.2 * size3))

x_train = np.concatenate((seq_vector_lab1[train_i1], seq_vector_lab2[train_i2], seq_vector_lab3[train_i3]))
y_train = np.concatenate((lab_vector_lab1[train_i1], lab_vector_lab2[train_i2], lab_vector_lab3[train_i3]))
x_val = np.concatenate((seq_vector_lab1[val_i1], seq_vector_lab2[val_i2], seq_vector_lab3[val_i3]))
y_val = np.concatenate((lab_vector_lab1[val_i1], lab_vector_lab2[val_i2], lab_vector_lab3[val_i3]))

#     sys.stdout.write(strftime("%Y-%m-%d %H:%M:%S") + ": Start training\n")
#     sys.stdout.flush()

In [7]:
print(x_train.shape, y_train.shape, x_val.shape, y_val.shape)

(92757, 200, 4) (92757, 3) (26502, 200, 4) (26502, 3)


In [8]:
def create_model(weights=''):
    #activation = keras.layers.LeakyReLU(alpha=0.3)
#     def activation(x):
#         return keras.activations.relu(x)
    activation = 'relu'
    ssInput = Input(shape = (200, len(classes)))
    ssConv = Conv1D(filters=256,
                    kernel_size=12,
                    padding = "valid",
                    activation=activation,
                    strides=1)(ssInput)
#     ssConv = activation(ssConv)
    ssPool = AveragePooling1D(pool_size = 5, strides = 5)(ssConv)
    ssDout1 = Dropout(rate=0.5)(ssPool)
    seqBiLstm = Bidirectional(LSTM(units = 128, return_sequences = True))(ssDout1)
    seqDout2 = Dropout(rate = 0.5)(seqBiLstm)
    ssFlat = Flatten()(seqDout2)
    ssDen1 = Dense(256, kernel_initializer='glorot_uniform', activation=activation)(ssFlat)
#     ssDen1 = activation(ssDen1)
    ssDout2 = Dropout(rate=0.5)(ssDen1)
    den2 = Dense(256, kernel_initializer = 'glorot_uniform', activation='relu')(ssDout2)
#     den2 = activation(den2)
    dout2 = Dropout(rate = 0.5)(den2)
    den3 = Dense(256, activation='relu')(dout2)
#     dens = activation(den3)
    den4 = Dense(3, activation = 'softmax')(den3)
    model = Model(inputs = ssInput, outputs = den4)



    adam = optimizers.Adam(lr=0.001)

    if weights:
        model.load_weights(weights)
        print("Created model and loaded weights from file: ", weights)
    else:
        print(model.summary())
        print("adam: 0.001", )
    model.compile(loss='categorical_crossentropy',
                  optimizer=adam,
                  metrics=['accuracy'])

    return model

In [9]:
weight_file = ''
model = create_model()
early_stop = EarlyStopping(monitor='val_loss', patience=10)
filepath = weight_file + "terminator.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True)

# load pretrained
# model = load_model('terminator.hdf5')

logs = model.fit(x_train, y_train, batch_size=128, verbose=1, epochs=100,
          validation_data=(x_val, y_val),
          callbacks=[checkpoint, early_stop])
    
#     sys.stdout.write(strftime("%Y-%m-%d %H:%M:%S") + ": Finished!\n")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 200, 4)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 189, 256)          12544     
_________________________________________________________________
average_pooling1d_1 (Average (None, 37, 256)           0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 37, 256)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 37, 256)           394240    
_________________________________________________________________
dropout_2 (Dropout)          (None, 37, 256)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9472)              0         
__________

In [10]:
# val_loss: 0.4679 - val_acc: 0.8073

In [11]:
model = load_model('terminator.hdf5')

In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 200, 4)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 189, 256)          12544     
_________________________________________________________________
average_pooling1d_1 (Average (None, 37, 256)           0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 37, 256)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 37, 256)           394240    
_________________________________________________________________
dropout_2 (Dropout)          (None, 37, 256)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9472)              0         
__________

In [13]:
y_val.shape

(26502, 3)

In [14]:
x_val.shape

(26502, 200, 4)

In [15]:
y_pred = model.predict(x_val)
print(classification_report(np.argmax(y_val, axis=-1), np.argmax(y_pred, axis=-1), target_names=[str(i) for i in range(3)]))

              precision    recall  f1-score   support

           0       0.92      0.96      0.94     10491
           1       0.69      0.30      0.42      5519
           2       0.72      0.91      0.80     10492

    accuracy                           0.80     26502
   macro avg       0.78      0.72      0.72     26502
weighted avg       0.79      0.80      0.78     26502



In [16]:
print(confusion_matrix(np.argmax(y_val, axis=-1), np.argmax(y_pred, axis=-1)))

[[10073    92   326]
 [  522  1663  3334]
 [  323   660  9509]]


In [17]:
y_pred_label = np.argmax(y_pred, axis=-1)
y_val_label = np.argmax(y_val, axis=-1)

In [18]:
y_pred_label[y_pred_label > 0] = 1
y_val_label[y_val_label > 0] = 1

In [19]:
y_pred_label = 1 - y_pred_label
y_val_label = 1- y_val_label

In [20]:
print(classification_report(y_val_label, y_pred_label))

              precision    recall  f1-score   support

           0       0.97      0.95      0.96     16011
           1       0.92      0.96      0.94     10491

    accuracy                           0.95     26502
   macro avg       0.95      0.95      0.95     26502
weighted avg       0.95      0.95      0.95     26502



In [21]:
y_pred_label.shape, 
y_val_label.shape

(26502,)

In [22]:
def sensitivy(y_pred, y_true):
    CM = confusion_matrix(y_true, y_pred)

    TN = CM[0][0]
    FN = CM[1][0]
    TP = CM[1][1]
    FP = CM[0][1]
    return TP / (FN + TP)
def acc(y_pred, y_true):
    CM = confusion_matrix(y_true, y_pred)

    TN = CM[0][0]
    FN = CM[1][0]
    TP = CM[1][1]
    FP = CM[0][1]
    
    return sklearn.metrics.accuracy_score(y_true, y_pred), (TP + TN) / (TP + TN + FN + FP)
def specificity(y_pred, y_true):
    CM = confusion_matrix(y_true, y_pred)

    TN = CM[0][0]
    FN = CM[1][0]
    TP = CM[1][1]
    FP = CM[0][1]
    return TN/(TN+FP)

In [23]:
acc(y_pred_label, y_val_label)

(0.9523432193796695, 0.9523432193796695)

In [24]:
sensitivy(y_pred_label, y_val_label)

0.9601563244685921

In [25]:
specificity(y_pred_label, y_val_label)

0.9472237836487415